# 目标检测

目标检测包含两个方面，一是图像识别，一个目标定位；

## 1.图片分类：

检测出图片包含的对象类型；

即输入一张图片到多层卷积网络，卷积网络输出特征向量，并通过sofmax单元预测图片类型

## 2.目标定位

比如图片检测出包含一辆汽车，进一步检测出汽车的坐标位置；

## 分类和定位的描述方式：

**输出类别** :用独热编码指示即可，比如有三个检测分类，行人，汽车，摩托车，用三位独热编码输出，最多有一个值类值为1；

**定位**：约定图片左上角为(0,0),右下角为(1,1),汽车的定位包含中心点，宽高，均为0～1的相对值；

![](./obj_detect01.jpg)

## 代价函数设计

添加一个$P_c$标志为，当检测到目的对象为1，否则为0；

所以根据检测结果，

$P_c$=0，忽略所有其他对象参数；

$P_c$=1，去所有参数的平方误差；

![](./obj_detect02.jpg)

注：对于上示代价函数，坐标参数及类别参数可以用平方误差函数，$P_c$参数可以用逻辑回归，当然用平方误差函数也可以；

# 特征点检测

神经网络可以通过输出图片上的特征点的坐标(目标定位)，来实现对目标特征的识别；

比如人脸检测应用中的定位眼角，鼻子，嘴巴等关键点；

若取多个特征点，比如足球运动员手肘，脚等取多个特征点，可以识别动作类型；

![](./obj03.jpg)

# 目标检测Object detection

## 1.滑动窗口的检测

创建一个标签训练集，也就是x和y表示适当剪切的汽车图片样本，训练一个汽车分类模型；

![](./obj04.jpg)

1.首先选定一个特定大小窗口，将窗口图片送入卷积网络，得到检测结果是否为汽车；

2.滑动该窗口，重新检测窗口图片是否为汽车；

3.继续滑动窗口，直至遍历完整个图片；总有一个窗口可以检测出图中汽车；

![](./obj000.png)

然后选择不同大小窗口，重复步骤1～3；
![](./obj05.jpg)

窗口滑动目标检测，明显的缺点是计算成本，因为图片被分为太多小方块，每个方块都要做一次卷积分类；并且无法预知窗口大小，没法精确计算目标位置；

因此传统的窗口滑动实现中，一般采用线性分类器，这样效率会更好；

## 卷积的滑动窗口实现

### 1.把神经网络的全连接层转换为卷积层

将5x5x16层，用400个5x5卷积核做卷积运算，得到1x1x400，再连接400个1x1卷积核，4个1x1卷积核，得到输出1x1x4

![](./obj06.jpg)

从数学上看是一样的，但是转换为卷积计算后，后续滑动过程中可以共用卷积结果

2.将多个窗口重复的检测运算，合并为一次卷积运算实现，即将需要划分的N个滑动窗口，对应为n个输出；


![](./obj10.jpg)


这样共享其中的重复卷积运算，提高运算效率，一次网络前向，计算出所有检测结果；

![](./obj11.jpg)

不过这个算法仍然有一个缺点，就是边界框的位置可能不够精确；

# 交并比

用来评价对象检测算法

计算两个边界框交集和并集之比

![](./obj20.jpg)

# 非极大抑制

实际应用中，图片可能花费为较小的方块，算法可能对同一个对象作出多次检测，如下图

![](./obj21.jpg)

多个方块都认为中心点落在内部，非极大抑制就是要去除这种重复性的检测；

![](./obj22.jpg)

非极大抑制，只输出概率最大的分类结果；

方法步骤：

1.过滤掉低于0.6的检测框，只保留置信度较高的检测框；

2.若还有剩余检测框，取概率最高的检测框；

3.将所有其他框跟这个概率最高框做过滤，去掉所有高IOU的框；

4.若存在，取剩下里，第二高概率的框，重复上述步骤，直到取完所有“高概率”框；

![](./obj23.jpg)

In [ ]:
非极大抑制的NMS实现过程中，交并比计算的分母采用两个框面积的最小值（min(Sa, Sb)）要优于两个框的并集面积（Sa+Sb-Sa&b）
特例说明：大框内部套小框，小框面积接近于阈值倍数